### Enhance underexposed image with contrast and brightness

there few technique i used to enhance an underexposed image. one of it is white balance correction. the purpose is to make the image looks more natural for their color casts. 

other than that i use LAB color space to seperate brighness from color information. this allow to enhance brightness and contrast without affecting colors directly. 

also i use CLAHE (Contrast Limited Adaptive Histogram Equalization) and apply it to the L-channel to enhance contrast adaptively, while preserving details.

then i raise the brightness and contrast to further enhance image visibility and details.

then i use bilateral filter to reduce noise while preserves edges.

In [27]:
import cv2
import numpy as np

# Apply automatic white balance correction
def white_balance(img):
    result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

def enhance_image(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path)

    img = white_balance(img)

    # Convert to LAB color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Split the LAB image into L, A, and B channels
    l, a, b = cv2.split(lab)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to L-channel
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(9,9))
    cl = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel with the original A and B channels
    limg = cv2.merge((cl,a,b))

    # Convert back to BGR color space
    enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    # Increase brightness
    brightness = 25
    enhanced = cv2.add(enhanced, brightness)

    # Increase contrast
    alpha = 1.1  # Contrast control (1.0-3.0)
    beta = 20    # Brightness control (0-100)
    enhanced = cv2.convertScaleAbs(enhanced, alpha=alpha, beta=beta)

    # Apply bilateral filter to reduce noise while preserving edges
    enhanced_img = cv2.bilateralFilter(enhanced, 1, 10, 10)

    # Save the result
    cv2.imwrite(output_path, enhanced_img)

    print(f"Enhanced image saved to {output_path}")

# Usage
input_image = "test_image_photo\\underexposed_photo.jpeg"
output_image = "enhanced_image_underexposed.jpg"
enhance_image(input_image, output_image)

Enhanced image saved to enhanced_image_underexposed.jpg


### Enhance overexposure image

for over exposure image, it is quite straight forward, as i just do reduce brightness and contrast to the image. 

In [224]:
from PIL import Image, ImageEnhance
import numpy as np

def reduce_overexposure(input_path, output_path, brightness_factor=0.9, contrast_factor=1.7):
    # Open the image
    img = Image.open(input_path)

    # Convert image to numpy array
    img_array = np.array(img)

    # Calculate the average brightness of the image
    average_brightness = np.mean(img_array)

    # Adjust brightness factor based on average brightness
    if average_brightness > 200:  # If the image is very bright
        brightness_factor *= 0.8  # Reduce brightness more aggressively

    # Adjust brightness
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(brightness_factor)

    # Adjust contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(contrast_factor)

    # Save the result
    img.save(output_path)

# Usage
input_image = "test_image_photo\overexposed_photo.jpeg"
output_image = "enhanced_image_overexposed.jpg"
reduce_overexposure(input_image, output_image)

### Enhance grainy image

for grainy image, i mainly focus on reducing the noise in the images. i use opencv library built in method, Non-local Mean Denoising. as it is effective algorithm to reduce grainess by averaging similar patches across image. 

then i touch up using gaussian blur to smoothen image and reduce remaining noise. after done removing noise and blur the image might appear flat, so i adjust a bit the brightness and contrast to make the image more detail.

In [32]:
import cv2
import numpy as np

def denoise_image(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply denoising
    denoised = cv2.fastNlMeansDenoising(img, None, h=10, searchWindowSize=21)
    
    # Apply Gaussian blur for further smoothing
    blurred = cv2.GaussianBlur(denoised, (3, 3), 0)
    
    # Adjust contrast and brightness
    alpha = 1.2 # Contrast control (1.0-3.0)
    beta = 10 # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(blurred, alpha=alpha, beta=beta)
    
    # Save the result
    cv2.imwrite(output_path, adjusted)

# Usage
denoise_image('test_image_photo\\grainy_image.jpg', 'enhanced_image_grainy.jpg')